<a href="https://colab.research.google.com/github/roo-21/lime-implementation/blob/main/lime-binary-nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tawfikelmetwally/employee-dataset
! unzip employee-dataset.zip

  0% 0.00/18.5k [00:00<?, ?B/s]
100% 18.5k/18.5k [00:00<00:00, 36.9MB/s]
Archive:  employee-dataset.zip
  inflating: Employee.csv            


In [ ]:
import pandas as pd
import numpy as np
import tensorflow
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
from keras import Sequential
from keras.layers import Dense,Dropout
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv("Employee.csv")
df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [ ]:
df.describe()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,LeaveOrNot
count,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,2015.062970,2.698259,29.393295,2.905652,0.343864
std,1.863377,0.561435,4.826087,1.558240,0.475047
min,2012.000000,1.000000,22.000000,0.000000,0.000000
25%,2013.000000,3.000000,26.000000,2.000000,0.000000
50%,2015.000000,3.000000,28.000000,3.000000,0.000000
75%,2017.000000,3.000000,32.000000,4.000000,1.000000
max,2018.000000,3.000000,41.000000,7.000000,1.000000


In [ ]:
encoded_df=pd.DataFrame()

In [ ]:
le=LabelEncoder()

In [ ]:
for column in df.drop(['Age',],axis=1).columns:
    encoded_df[column]=le.fit_transform(df[column]) #all columns except Age is converted to categorical labels
                                                    #JoininYear is converted to categorical becase there are only 7 years from 2012-2018
                                                    # (see min and max under JoiningYear in describe)

In [ ]:
def group_ages(x): #function to bin ages
    if x<=25:
        return 0
    elif x<=30:
        return 1
    elif x<=35:
        return 2
    elif x<=40:
        return 3
    else:
        return 4

encoded_df['Age']= df['Age'].apply(group_ages)

In [ ]:
encoded_df #new encoded dataset

,Education,JoiningYear,City,PaymentTier,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,Age
0,0,5,0,2,1,0,0,0,2
1,0,1,2,0,0,0,3,1,1
2,0,2,1,2,0,0,2,0,3
3,1,4,0,2,1,0,5,1,1
4,1,5,2,2,1,1,2,1,0
...,...,...,...,...,...,...,...,...,...
4648,0,1,0,2,0,0,4,0,1
4649,1,1,2,1,1,0,2,1,3
4650,1,6,1,2,1,0,5,1,1
4651,0,0,0,2,1,1,2,0,1


In [ ]:
encoded_df.describe()

,Education,JoiningYear,City,PaymentTier,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot,Age
count,4653.00000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000,4653.000000
mean,0.26456,3.062970,0.793681,1.698259,0.597034,0.102729,2.905652,0.343864,1.289706
std,0.52112,1.863377,0.841979,0.561435,0.490547,0.303638,1.558240,0.475047,0.995635
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.00000,1.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,1.000000
50%,0.00000,3.000000,1.000000,2.000000,1.000000,0.000000,3.000000,0.000000,1.000000
75%,0.00000,5.000000,2.000000,2.000000,1.000000,0.000000,4.000000,1.000000,2.000000
max,2.00000,6.000000,2.000000,2.000000,1.000000,1.000000,7.000000,1.000000,4.000000


In [ ]:
# preparing input data as numpy arrays
x=encoded_df.drop('LeaveOrNot',axis=1).values
y=encoded_df['LeaveOrNot'].values.reshape(-1,1)

In [ ]:
# splitting train and test data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2, random_state=1)

In [ ]:
# Binary classification network
model=Sequential()
model.add(Dense(64,input_shape=(8,),activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=50,verbose=1)

Epoch 1/50
117/117 [==============================] - 1s 2ms/step - loss: 0.5965 - accuracy: 0.6862
Epoch 2/50
117/117 [==============================] - 0s 2ms/step - loss: 0.5648 - accuracy: 0.7230
Epoch 3/50
117/117 [==============================] - 0s 2ms/step - loss: 0.5518 - accuracy: 0.7340
Epoch 4/50
117/117 [==============================] - 0s 2ms/step - loss: 0.5351 - accuracy: 0.7458
Epoch 5/50
117/117 [==============================] - 0s 3ms/step - loss: 0.5161 - accuracy: 0.7558
Epoch 6/50
117/117 [==============================] - 0s 4ms/step - loss: 0.4985 - accuracy: 0.7676
Epoch 7/50
117/117 [==============================] - 1s 6ms/step - loss: 0.4843 - accuracy: 0.7786
Epoch 8/50
117/117 [==============================] - 1s 9ms/step - loss: 0.4760 - accuracy: 0.7845
Epoch 9/50
117/117 [==============================] - 1s 8ms/step - loss: 0.4656 - accuracy: 0.7966
Epoch 10/50
117/117 [==============================] - 1s 5ms/step - loss: 0.4547 - accuracy: 0.8028

In [ ]:
def prediction(x):
    predicted=model.predict(x)
    return predicted

In [ ]:
y_pred = prediction(x_test)

30/30 [==============================] - 0s 3ms/step


In [ ]:
accuracy_score((y_pred>.5).astype(np.int64),y_test)

0.8259935553168636

In [ ]:
! pip install -q lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import lime
from lime.lime_tabular import LimeTabularExplainer

In [ ]:
# creating explainer object
explainer = LimeTabularExplainer(x_train, mode = 'classification', training_labels = y_train, verbose=True,discretize_continuous=True,feature_names=list(x),class_names=[0,1])

In [ ]:
# getting local explanation of 5th row
j = 5
exp = explainer.explain_instance(x_test[j], prediction, num_features=8,top_labels=1)

157/157 [==============================] - 1s 4ms/step
Intercept 0.7053759589942195
Prediction_local [0.44475229]
Right: 0.15693106


In [ ]:
j = 2
exp = explainer.explain_instance(x_test[j], prediction, num_features=8,top_labels=1)

157/157 [==============================] - 1s 5ms/step
Intercept 0.6781845797431872
Prediction_local [0.46384942]
Right: 0.14862537
